In [ ]:
import os
import papermill as pm
from logger import logger

## Generate Embeddings

In [ ]:
notebook = "embeddings.ipynb"
pm.inspect_notebook(notebook)

In [ ]:
# model_name = "all-MiniLM-L6-v2"
# model_name = "all-mpnet-base-v2"
model_name = "bge-large-en-v1.5"
owner_name = 'BAAI'

for strategy in ["mean"]:
    logger.info(
        f"Running {notebook} for {model_name} model with {strategy} pooling strategy"
    )
    pm.execute_notebook(
        input_path="embeddings.ipynb",
        output_path=os.path.join(
            "..",
            "artifacts",
            "notebooks",
            f'{model_name}_{strategy}_emb.ipynb',
        ),
        parameters={"MODEL_NAME": model_name, "POOLING_STRATEGY": strategy, "OWNER":owner_name},
    )

## Evaluate Similarities

In [ ]:
notebook = "similarity.ipynb"
pm.inspect_notebook(notebook)

In [ ]:
# model_name = "all-MiniLM-L6-v2"
# model_name = "all-mpnet-base-v2"

for strategy in ["mean"]:
    logger.info(
        f"Running {notebook} for {model_name} model with {strategy} pooling strategy"
    )
    pm.execute_notebook(
        input_path="similarity.ipynb",
        output_path=os.path.join(
            "..",
            "artifacts",
            "notebooks",
            f"{model_name}_{strategy}_sim.ipynb",
        ),
        parameters={"MODEL_NAME": model_name, "POOLING_STRATEGY": strategy,"OWNER":owner_name},
    )